In [1]:
import json
import numpy as np
import torch
from tqdm import tqdm

from src.models.high_level_model import HighLevelModel
from src.data.multi_output_dataset import MultiOutputDataModule
from src.calibration.nonconformity_functions import NONCONFORMITY_FN_DIC
from src.calibration.calibration import CALIBRATION_FN_HIGH_DIC
from src.models.conformal_prediction import standard_prediction, clustered_prediction
from src.metrics import (
    compute_overall_efficiency,
    compute_overall_informativeness,
    compute_taskwise_informativeness,
    compute_taskwise_efficiency,
)

In [2]:
MDC_CLASSES = [12, 11]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = HighLevelModel.load_from_checkpoint(
    "models/mdc-high-level-model.ckpt",
    task_num_classes=MDC_CLASSES,
    map_location=device,
)
model = model.eval()

data = MultiOutputDataModule(
    root_dir="data/", batch_size=64, num_workers=0, task_num_classes=MDC_CLASSES
)
data.setup()

In [4]:
with open("models/mdc-high-level-calibration.json", "r") as file:
    calibration_data = json.load(file)

In [5]:
def generate_predictions(model, dataloader):
    y_preds = []
    y_trues = []

    for batch in tqdm(dataloader):
        x, y = batch[0].to(device), batch[1].to(device)
        with torch.no_grad():
            pred = model.predict_step(x)  # List of T tensors, each (B, C_t)
        pred = [p.cpu() for p in pred]  # Move to CPU

        batch_size = pred[0].shape[0]
        for i in range(batch_size):
            sample_preds = [p[i] for p in pred]  # T predictions for sample i
            y_preds.append(sample_preds)
        y_trues.extend(y.cpu().numpy())  # Each is shape (T,)

    return y_preds, y_trues

In [6]:
def test_calibration_from_preds(
    y_preds: list,
    y_trues: list,
    nonconformity_fn: str,
    calibration_type: str,
    calibration_data: dict,
    task_num_classes: list,
):
    """
    Evaluate conformal calibration using precomputed predictions for multi-task classification.

    This function computes nonconformity scores, applies the specified conformal calibration
    method, and evaluates prediction sets based on coverage, efficiency, and informativeness.

    Args:
        y_preds (list): List of B samples, each is a list of T tensors/logits of shape (C_t,).
        y_trues (list): List of B samples, each is a list of T true labels (ints).
        nonconformity_fn (str): Name of the nonconformity function to use (e.g., 'hinge', 'margin').
        calibration_type (str): Type of calibration to apply ('scp_task_thresholds', 'ccp_cluster_thresholds', etc.).
        calibration_data (dict): Dictionary containing thresholds or cluster mappings for each method.
        task_num_classes (list): List of ints representing number of classes for each task.

    Returns:
        None: Prints taskwise and overall coverage, efficiency, and informativeness statistics.
    """
    T = len(task_num_classes)
    B = len(y_preds)

    # Reshape y_preds: list[B][T][C_t] --> list[T][B][C_t]
    y_preds_by_task = [
        np.stack([y_preds[b][t] for b in range(B)])
        for t in range(T)
    ]

    # Compute nonconformity scores
    nonconformity_scores = NONCONFORMITY_FN_DIC[nonconformity_fn](y_preds_by_task)

    clustered = "cluster" in calibration_type
    prediction = (
        clustered_prediction(
            nonconformity_scores,
            calibration_data[nonconformity_fn][calibration_type],
        )
        if clustered
        else standard_prediction(
            nonconformity_scores,
            calibration_data[nonconformity_fn][calibration_type],
        )
    )  # list[T][B], each element is prediction set for one task and sample

    # Reshape predictions: list[T][B] --> list[B][T]
    predictions_by_sample = list(zip(*prediction))

    # Evaluate coverage
    in_it = np.zeros(T, dtype=int)
    for t in range(T):
        for i in range(B):
            if y_trues[i][t] in predictions_by_sample[i][t]:
                in_it[t] += 1

    overall_eff = compute_overall_efficiency(prediction)
    taskwise_eff = compute_taskwise_efficiency(prediction)
    overall_info = compute_overall_informativeness(prediction)
    taskwise_info = compute_taskwise_informativeness(prediction)

    # Reporting
    print("Accuracies of the calibrated method:")
    for t in range(T):
        print(f"Accuracy of Task {t}: {in_it[t]} / {B} = {in_it[t] / B:.2%}")
    overall = sum(in_it)
    total = B * T
    print(f"Overall: {overall} / {total} = {overall / total:.2%}")
    for t in range(T):
        print(f"Efficiency of Task {t}: {taskwise_eff[t]:.4f}")
    print(f"Overall Efficiency: {overall_eff:.4f}")
    for t in range(T):
        print(f"Informativeness of Task {t}: {taskwise_info[t]:.4f}")
    print(f"Overall Informativeness: {overall_info:.4f}")

In [7]:
y_preds, y_trues = generate_predictions(model, data.test_dataloader())

100%|██████████| 76/76 [02:08<00:00,  1.69s/it]


In [8]:
for nonconformity_fn in NONCONFORMITY_FN_DIC.keys():
    print("-------------------------------------------------")
    print(f"Nonconformity function: {nonconformity_fn}")
    for calibration_type in CALIBRATION_FN_HIGH_DIC.keys():
        print("-------------------------------------------------")
        print(f"Calibration type: {calibration_type}")
        test_calibration_from_preds(
            y_preds,
            y_trues,
            nonconformity_fn,
            calibration_type,
            calibration_data,
            MDC_CLASSES,
        )

-------------------------------------------------
Nonconformity function: hinge
-------------------------------------------------
Calibration type: scp_global_threshold
Accuracies of the calibrated method:
Accuracy of Task 0: 4553 / 4812 = 94.62%
Accuracy of Task 1: 4573 / 4812 = 95.03%
Overall: 9126 / 9624 = 94.83%
Efficiency of Task 0: 1.1367
Efficiency of Task 1: 1.1338
Overall Efficiency: 1.1353
Informativeness of Task 0: 0.8664
Informativeness of Task 1: 0.8732
Overall Informativeness: 0.8698
-------------------------------------------------
Calibration type: scp_task_thresholds
Accuracies of the calibrated method:
Accuracy of Task 0: 4557 / 4812 = 94.70%
Accuracy of Task 1: 4560 / 4812 = 94.76%
Overall: 9117 / 9624 = 94.73%
Efficiency of Task 0: 1.1438
Efficiency of Task 1: 1.1214
Overall Efficiency: 1.1326
Informativeness of Task 0: 0.8606
Informativeness of Task 1: 0.8836
Overall Informativeness: 0.8721
-------------------------------------------------
Calibration type: ccp_cla